In [ ]:
# --------------------------------------------------------------------------
# In order to run, this script needs the following 3rd party libraries
# /// script
# requires-python = ">=3.8"
# dependencies = [
#   "stubber>=1.24.0",
# ]
# ///
# PEP723: https://peps.python.org/pep-0723/
# --------------------------------------------------------------------------

In [13]:
import shutil
from pathlib import Path
from typing import Optional


# version = "v1.23.0"
version = "preview"
version = "v1.24.1"

# merge / build
# fmt: off
port = "rp2" ; board = "RPI_PICO"
port = "all" ; board = "all"
port = "esp32" ; board = "all"
# fmt: on

if version == "preview":
    flat_version = "v1_25_0_preview"
else:
    flat_version = version.replace(".", "_")

docstubs = Path(f"repos/micropython-stubs/stubs/micropython-{flat_version}-docstubs")
reference = Path("repos/micropython-stubs/micropython-reference")


def cp_stubmod(module: str, target: Optional[Path] = None) -> None:
    target = target or docstubs
    src = reference / module
    dst = target / module
    return shutil.copytree(src, dst, dirs_exist_ok=True)


def cp_stubfile(module: str, target: Optional[Path] = None) -> None:
    target = target or docstubs
    if target.is_dir():
        target.mkdir(parents=True, exist_ok=True)
    src = reference / module
    dst = target / module
    return shutil.copy2(src, dst)

In [ ]:
!stubber switch {version}

In [ ]:
! cd d:\mypython\micropython-stubber
!stubber docstubs --enrich 


In [ ]:
import os
cwd = os.getcwd()
os.chdir("d:\\mypython\\micropython-stubber\\repos\\micropython-stubs\\publish\\micropython-stdlib-stubs")

%run build.py

os.chdir(cwd)

In [ ]:

!stubber frozen --version {version}

^C


21:42:10 | ℹ️ micropython-stubber 1.24.0
21:42:10 | ℹ️ fetch updates
21:42:12 | ℹ️ Switching to v1.24.1
21:42:18 | ℹ️ checkout arduino-lib: 0
21:42:19 | ℹ️ repos\micropython                        v1.24.1
21:42:19 | ℹ️ repos\micropython-lib                    v1.24.0
21:42:19 | ℹ️ repos\micropython\lib\micropython-lib    v1.24.0
21:42:19 | ℹ️ MicroPython version : v1.24.1
21:42:19 | ℹ️ looking for manifests in  
d:\mypython\micropython-stubber\repos\micropython
21:42:19 | ℹ️ manifests found: 73
21:42:19 | ℹ️ manifests: 73
21:42:20 | ℹ️ port-board: esp32
21:42:20 | ℹ️ port-board: esp32-ARDUINO_NANO_ESP32
21:42:20 | ⚠️ HACK- Adding [TOP]/lib/arduino-lib to paths: ARDUINO_NANO_ESP32
21:42:21 | ℹ️ port-board: esp32-LILYGO_TTGO_LORA32
21:42:21 | ℹ️ port-board: esp32-LOLIN_C3_MINI
21:42:21 | ℹ️ port-board: esp32-LOLIN_S2_MINI
21:42:21 | ℹ️ port-board: esp32-LOLIN_S2_PICO
21:42:21 | ℹ️ port-board: esp32-M5STACK_ATOM
21:42:21 | ℹ️ port-board: esp32-UM_FEATHERS2
21:42:21 | ℹ️ port-board: esp32-

Traceback (most recent call last):
  File "d:\mypython\micropython-stubber\.venv\Scripts\\stubber", line 6, in <module>
    sys.exit(stubber_cli())
             ^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\rich_click\rich_command.py", line 367, in __call__
    return super().__call__(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\click\core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\rich_click\rich_command.py", line 152, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\click\core.py", line 1728, in invoke
    rv.append(sub_ctx.command.invoke(sub_ctx))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\mypython\micropython-stubber\.venv\Lib\site-packages\click\core.py", line

In [ ]:

!stubber merge  --port {port} --board {board} --version {version}
# LFU - 1:20
# LRU - 1:20
# None 

In [ ]:
!stubber build  --port {port} --board {board}  --version {version}

In [ ]:
# invalidate the cached stubs to force a reload for testing
import shutil

shutil.rmtree("repos/micropython-stubs/.pytest_cache/", ignore_errors=True)
shutil.rmtree("repos/micropython-stubs/tests/quality_tests/.pytest_cache", ignore_errors=True)

In [ ]:
# !stubber publish  --port {port} --board {board}  --version {version}